In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from torch.utils.data import DataLoader, Dataset
import torch
import trimesh
from functools import partial

from Solvers import SolverEmbedding,Models,Loss

from dataEmbedding.dataEmbedding import Read_Load_BuildBatch
from dataEmbedding.dataEmbeddingLoader import GenerateDataLoader,check_dataset,collate_embedding
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\michalBrysiak\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\michalBrysiak\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\michalBrysiak\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\michalBrysiak\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
def vizualizer_stanford_data(vox_color):
    vox_color = np.rollaxis(vox_color, 0,4)
    ind=vox_color[:,:,:]!=[0,0,0,0]
    vox = ind[:, :, :,0]
    z=trimesh.voxel.VoxelGrid(vox)
    l=z.as_boxes(colors=vox_color)
    return l.show()

In [3]:
batchSize=64
#primiData = Read_Load_BuildBatch('/primitives',batchSize,'primitives')
stanData=Read_Load_BuildBatch('nrrd_256_filter_div_32_solid/nrrd_256_filter_div_32_solid',batchSize,'stanford_data',"captions.tablechair.csv")


In [4]:
train_dataset = GenerateDataLoader(stanData.data_agg_train,'stanford_data',stanData.data_dir,stanData.labelEnc,stanData.dict_word2idx,'train')

val_dataset = GenerateDataLoader(stanData.data_agg_val,'stanford_data',stanData.data_dir,stanData.labelEnc,stanData.dict_word2idx,'val')

test_dataset=GenerateDataLoader(stanData.data_agg_test,'stanford_data',stanData.data_dir,stanData.labelEnc,stanData.dict_word2idx,'test')

dataloader = {
            'train': DataLoader(
                train_dataset, 
                batch_size=batchSize * 2,              
                drop_last=check_dataset(train_dataset, batchSize * 2),
                collate_fn=collate_embedding,
                num_workers=4
            ),
            'val': DataLoader(
                val_dataset, 
                batch_size=batchSize*2,
                collate_fn=collate_embedding,
                num_workers=4
            ),
            'test': DataLoader(
                test_dataset, 
                batch_size=batchSize*2,
                collate_fn=collate_embedding
                #num_workers=2
            )
    }       


In [5]:
for iter,(_,_,labels,texts,_,shapes) in enumerate(dataloader['train']):
    
    batch_size = shapes.size(0)
    texts = texts.to(device)
    text_labels = labels.to(device)

    shapes = shapes.to(device).index_select(0, torch.LongTensor([i * 2 for i in range(batch_size // 2)]).to(device))
    shape_labels = labels.to(device).index_select(0, torch.LongTensor([i * 2 for i in range(batch_size // 2)]).to(device))
        

    #s = ShapeModel(shapes)
    #t = TextModel(texts)

    print(texts.shape)

    if iter==5:
      break

torch.Size([128, 19])
torch.Size([128, 20])
torch.Size([128, 20])
torch.Size([128, 20])
torch.Size([128, 20])
torch.Size([128, 20])


In [5]:
criterion={
        'walker': Loss.RoundTripLoss(weight=1.0),
        'visit': Loss.AssociationLoss(weight=0.25),
        'metric': Loss.SmoothedMetricLoss(margin=1.0)
        }

In [6]:
ShapeModel=Models.ShapeEncoder()
ShapeModel=ShapeModel.to(device)
TextModel=Models.TextEncoder(len(stanData.dict_word2idx))
TextModel=TextModel.to(device)
optimizer = torch.optim.Adam(list(ShapeModel.parameters()) + list(TextModel.parameters()), lr=0.001, weight_decay=0.0001)
history=SolverEmbedding.Solver(TextModel,ShapeModel,dataloader,optimizer,criterion,128)

In [7]:
history.train(16)

epoch [1/16] starting...



  0%|          | 1/1963 [00:23<12:41:54, 23.30s/it]

62.49559020996094


  5%|▌         | 101/1963 [00:56<10:31,  2.95it/s] 

57.434200286865234


...

 97%|█████████▋| 1901/1963 [09:43<00:20,  3.08it/s]

54.956298828125


100%|██████████| 1963/1963 [10:03<00:00,  3.25it/s]


validating...



  0%|          | 0/200 [00:00<?, ?it/s]